### Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

В датафрейме 21525 строк, 12 столбцов.
Описание столбцов:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита <br> 
2174 пропущенных значения в столбце days_employed (общий трудовой стаж в днях), так же есть отрицательные значения. И
2174 пропущенных значения в столбце total_income (ежемесячный доход).

### Шаг 2. Предобработка данных

### Обработка пропусков

Пропуски обнаружены в столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход). Т.к. столбец days_employed не пригодится для анализа в рамках данного проекта, то пропуски в этом столбце заменять не будем <br>
Рассмотрим значения в столбце total_income:

In [3]:
data['total_income'].describe().astype('int')

count      19351
mean      167422
std       102971
min        20667
25%       103053
50%       145017
75%       203435
max      2265604
Name: total_income, dtype: int64

Т.к. в данных видно большую неравномерность (большое значение std) и пропущенных данных достаточно много и данный столбец необходим нам для анализа, то для заполнения пропусков воспользуемся медианным значением.

In [4]:
median_income = data['total_income'].median()  # получение медианы ежемесячного дохода
data['total_income'] = data['total_income'].fillna(median_income)

Также рассмотрим другие столбцы с количественными данными, необходимыми для анализа и оценим их верность:

In [5]:
data['children'].describe().astype('int')

count    21525
mean         0
std          1
min         -1
25%          0
50%          0
75%          1
max         20
Name: children, dtype: int64

Есть данные с указанием количества детей -1 и 20.

In [6]:
print('Количество записей с указанием количества детей 20:', data[data['children'] == 20].shape[0])
print('Количество записей с указанием количества детей -1:', data[data['children'] == -1].shape[0])

Количество записей с указанием количества детей 20: 76
Количество записей с указанием количества детей -1: 47


Количество детей -1 можно заменить на 1. Т.к. наиболее вероятно, что "-" возник при вводе единицы. <br>
Записи c количеством детей 20 удалим. Т.к. их не много от общего числа и природа этой ошибки не ясна. Возможно это опечатка при вводе цифры 2 (тогда почему нет опечаток 10 или 3.?) или при указании количества детей был указан возраст одного единственного (тогда почему нет других больших значений?)

In [7]:
data.loc[data['children'] == -1, 'children'] = 1
data = data[data['children'] != 20]
data['children'].describe().astype('int')

count    21449
mean         0
std          0
min          0
25%          0
50%          0
75%          1
max          5
Name: children, dtype: int64

### Вывод

Было найдено 2174 пропущенных значения в столбце days_employed (общий трудовой стаж в днях), так же есть отрицательные значения (но это не повлияет на полученные выводы) и 2174 пропущенных значения в total_income (ежемесячный доход).<br>
Возможно пропуски в столбце total_income появились при получении информации от работодателя (не были внесены в БД после получения), т.к. пропущенные значения в этом столбце совпадают с пропусками в столбце days_employed.<br>
Пропуски имеют тип "NaN".<br>
Т.к. необходимо будет решить, есть ли зависимость между уровнем дохода и возвратом кредита в срок, и значения дохода имеют большой разброс, то пропущенные значения в столбце "ежемесячынй доход" заменены на медиану этого столбца.<br>
Так же были замечены аномальные данные в столбце 'children'<br>
Значения -1 были заменены на 1 (т.к. вероятно -1 это опечатка при записи 1), а значения - 20 удалены (т.к. не понятно это опечатка при вводе цифры 2 или вместо количества детей был указан возраст)

### Замена типа данных

In [8]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21449 entries, 0 to 21524
Data columns (total 12 columns):
children            21449 non-null int64
days_employed       19284 non-null float64
dob_years           21449 non-null int64
education           21449 non-null object
education_id        21449 non-null int64
family_status       21449 non-null object
family_status_id    21449 non-null int64
gender              21449 non-null object
income_type         21449 non-null object
debt                21449 non-null int64
total_income        21449 non-null int64
purpose             21449 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Вывод

Выполнена замена значений в столбце total_income из float на int методом .astype, т.к. значение было уже числовым, astype сделал его целочисленным

### Обработка дубликатов

In [9]:
print('количество дубликатов (до замены):', data.duplicated().sum())
data = data.drop_duplicates()
print('количество дубликатов (после замены):', data.duplicated().sum())

количество дубликатов (до замены): 54
количество дубликатов (после замены): 0


### Вывод

Т.к. у клиентов банка нет уникального идентификатора, считаем полностью совпадающие строки дубликатами, т.к. очень маловероятно полное совпадение двух клиентов (в т.ч. количество дней стажа и ежемесячный доход). 
Возможно дубликаты появились про копировании информации.
Выполнено удаление дубликатов

### Лемматизация

In [10]:
from pymystem3 import Mystem
m = Mystem()
lemma = []
for purpose in data['purpose']:
    lemma.append(m.lemmatize(purpose))
data.loc[:, 'purpose_lemma'] = lemma
data.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:489: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val = np.array(val, copy=False)
/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


### Вывод

Для каждого значения в столбце 'purpose' получил лемму (список из нескольких слов) и поместил их в отдельный столбец 'purpose_lemma'

### Категоризация данных

In [11]:
# добавим 2 столбца - зарплата выше/ниже среднего и "id цели кредита"

mean = data['total_income'].mean()
def income_mean(income): # зарплата выше или ниже средней
    if income < mean:
        return 'ниже среднего'
    else:
        return 'выше среднего'

purpose = {'автомобиль': 0, 'свадьба': 1, 'недвижимость': 2, 'жилье': 2, 'образование': 3}
def purpose_id (purpose_lemma):
    for key in purpose:
        if key in purpose_lemma:
            return purpose[key]

data['income_level'] = data['total_income'].apply(income_mean)
data['purpose_id'] = data['purpose_lemma'].apply(purpose_id)


children_income_debt = data.pivot_table(index=['income_level'], columns='children', values='debt', aggfunc='mean')

# словарь семейного положения
family_status_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

# таблица family_status_debt / процент должников
family_status_debt = (data[['family_status_id', 'debt']].groupby('family_status_id').sum() / data[['family_status_id', 'debt']].groupby('family_status_id').count())*100


# таблица цель кредита / процент должников
purpose_debt = (data[['purpose_id', 'debt']].groupby('purpose_id').sum() / data[['purpose_id', 'debt']].groupby('purpose_id').count())*100



### Вывод

Выполнил группировку по наличию детей и посчитал процент невозврата кредита среди них отнормировав значения.
Создал словарь семейное положение - id. Выполнил группировку по семейному положению и так жж посчитал процент невозврата кредита среди них. Далее разделил размер зарплаты на два уровня - выше среднего и ниже. И так же сргупприовал всех клиентов по этому признаку и посчитал процент невозврата.
Касательно целей взятия кредита - оценил частоту встречаемых лемм и аналогично сгруппировал по целям, присвоив им id

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
children_income_debt

children,0,1,2,3,4,5
income_level,,,,,,
выше среднего,0.071371,0.088219,0.091892,0.058824,0.166667,0.0
ниже среднего,0.077521,0.093638,0.096037,0.097938,0.068966,0.0


### Вывод

Из таблицы видно, что между наличием детей и возвратом кредита в срок есть зависимость. Причём, есть зависимость так же и от дохода. С ростом количества детей - растёт влияние ежемесячного дохода на вероятность выплатить кредит в срок. Среди людей с отсутсвтием детей всё же чуть меньше должников по кредитам. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
family_status_debt.merge(family_status_dict, on='family_status_id')

,family_status_id,debt,family_status
0,0,7.547784,женат / замужем
1,1,9.274874,гражданский брак
2,2,6.596859,вдовец / вдова
3,3,7.041073,в разводе
4,4,9.746519,Не женат / не замужем


### Вывод

Люди не женатые / не замужем или живущие гражданским браком чаще являются задолжниками по кредитам. Те кто находятся или находились в браке - более ответственно относятся к кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
children_income_debt

children,0,1,2,3,4,5
income_level,,,,,,
выше среднего,0.071371,0.088219,0.091892,0.058824,0.166667,0.0
ниже среднего,0.077521,0.093638,0.096037,0.097938,0.068966,0.0


### Вывод

Из таблицы видно, что зависимость между уровнем дохода и возвратом кредита в срок есть. Причём зависимость выражена по разному в зависимости от количества детей. Так бездетные и те у кого 1, 2 или 3 ребенка примерно одинаково возвращают кредиты (те, у кого уровень дохода ниже среднего, немного чаще являются должниками). А среди тех, у кого 3 или 4 ребенка, должниками чаще являются те, у кого уровень доходов выше среднего.

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
print(purpose_debt)
print(purpose)

                debt
purpose_id          
0           9.342964
1           7.867584
2           7.236964
3           9.227307
{'автомобиль': 0, 'свадьба': 1, 'недвижимость': 2, 'жилье': 2, 'образование': 3}


### Вывод

Меньше всего должников из тех, кто берёт кредит на недвижимость, т.к. потерять недвижимость при неуплате кредита будет большой проблемой. Больше всего неплательщиков среди тех, кто берёт кредиты на автомобили и образование. 

### Шаг 4. Общий вывод

В данном проекте исследовали влияние различных факторов на факт погашения кредита в срок. Видно, что почти все из указанных факторов так или иначе влияют и связаны с платежеспособностью клиента.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.